ナップサック問題問題

入れられる品物の重さの合計に制限があるナップサックがある．それぞれ重さと値段がついた品物がn個あるとき，ナップサックに入るものの価値を最大にする組み合わせを求める．

In [1]:
#品物のリストを作る
from collections import namedtuple
import random

random.seed(7)
Item = namedtuple("Item", ("name", "weight", "price"))
num = 20
item_list = []
max_weight = 5
max_price = 50

price_list = list(range(1, max_price+1))
random.shuffle(price_list)

for i in range(num):
  w = random.randint(1, max_weight)
  item = Item(i, w, price_list.pop())
  item_list.append(item)

In [2]:
item_list

[Item(name=0, weight=3, price=21),
 Item(name=1, weight=1, price=10),
 Item(name=2, weight=5, price=26),
 Item(name=3, weight=1, price=42),
 Item(name=4, weight=5, price=4),
 Item(name=5, weight=1, price=5),
 Item(name=6, weight=5, price=35),
 Item(name=7, weight=2, price=7),
 Item(name=8, weight=4, price=24),
 Item(name=9, weight=5, price=38),
 Item(name=10, weight=4, price=46),
 Item(name=11, weight=3, price=33),
 Item(name=12, weight=4, price=14),
 Item(name=13, weight=5, price=3),
 Item(name=14, weight=4, price=6),
 Item(name=15, weight=3, price=28),
 Item(name=16, weight=3, price=27),
 Item(name=17, weight=2, price=45),
 Item(name=18, weight=2, price=16),
 Item(name=19, weight=2, price=37)]

In [16]:
#knapsackクラス
class Knapsack:
  def __init__(self, size):
    self.size = size
    self.weight = 0
    self.value = 0
    self.items = []

  def append(self, item):
    #ナップサックにアイテムを追加する
    if not self.has_room_for(item):
      raise ValueError("個のアイテムは入れられません．重量オーバーですです")
    self.items.append(item)
    self.weight += item.weight
    self.value += item.price

  def has_room_for(self, item):
    return self.size >= self.weight + item.weight
  
  def __str__(self):
    val = "重さ{}kg / 価格{}万円".format(self.weight, self.value)
    return val

貪欲法

値段を重さで割って降順にソートし，上から順に入るだけ詰めていく．

In [4]:
def greedy(item, size_limit):
  sorted_item_list = sorted(item, key=lambda x: x.price/x.weight, reverse=True)
  my_knapsack = Knapsack(size_limit)
  for v in sorted_item_list:
    #入る余地があるなら品物を入れる
    try:
      my_knapsack.append(v)
    except ValueError:
      continue
  return my_knapsack

In [7]:
knap_g = greedy(item_list, 40)
print(knap_g)

重さ39kg / 価格407万円


In [8]:
knap_g.items

[Item(name=3, weight=1, price=42),
 Item(name=17, weight=2, price=45),
 Item(name=19, weight=2, price=37),
 Item(name=10, weight=4, price=46),
 Item(name=11, weight=3, price=33),
 Item(name=1, weight=1, price=10),
 Item(name=15, weight=3, price=28),
 Item(name=16, weight=3, price=27),
 Item(name=18, weight=2, price=16),
 Item(name=9, weight=5, price=38),
 Item(name=0, weight=3, price=21),
 Item(name=6, weight=5, price=35),
 Item(name=8, weight=4, price=24),
 Item(name=5, weight=1, price=5)]

力づくの方法

20個の品物がナップサックに入るかどうかの組み合わせは2^20個ある．これを全て試す．

組合せ爆発：問題の入力サイズnに対して可能な解の個数が指数関数や階乗など，多項式と比べて爆発的に増えていく状態の総称

→組み合わせ爆発が起こるような問題については，最適解を諦め最適解の良い近似を求める近似アルゴリズムを使う

In [14]:
import itertools

def brute_force(items, size_limit):
  candidate = None
  for pattern in itertools.product((0, 1), repeat=len(items)):
    my_box = []
    for i, val in enumerate(pattern):
      if val:
        my_box.append(item_list[i])
    w = sum([item.weight for item in my_box])
    if w > size_limit:
      continue
    value = sum([item.price for item in my_box])
    if candidate is None or value > candidate.value:
      knapsack = Knapsack(size_limit)
      for v in my_box:
        knapsack.append(v)
      candidate = knapsack
  return candidate

In [15]:
knap_bf = brute_force(item_list, 40)
print(knap_bf)

重さ40kg / 価格409万円


ナップサック問題での最適解

動的計画法を用いた擬多項式時間アルゴリズムと呼ばれる方法で最適解を求めることができる

動的計画法による計算時間：

品物の数をn，ナップサックの容量をmとした場合の計算は(n+1)*(m=1)の表を更新することに等しい．従って計算量はO(nm)

In [22]:
#動的計画法を使った実装
def dp(items, size_limit):
    n = len(items)
    # 価値を記録する表を作成 （行が品物、列が許容サイズ）
    table = [[0]*(size_limit+1) for i in range(n+1)]
    # 価値を更新したかどうかを記録するための表
    flag = [[False]*(size_limit+1) for i in range(n+1)]
    for i in range(1, n+1):
        target = items[i-1]
        w = target.weight
        for j in range(1, size_limit+1):
            yellow = table[i-1][j]
            table[i][j] = yellow
            if w > j: continue
            pink = table[i-1][j-w]
            include_this = target.price + pink
            table[i][j] = max(yellow, include_this)
            flag[i][j] = include_this > yellow
    i = n
    j = size_limit
    my_knapsack = Knapsack(size_limit)
    while i > 0 and j > 0:
        if flag[i][j]:
            my_knapsack.append(items[i-1])
            j -= items[i-1].weight
        i -= 1
    return my_knapsack

In [23]:
knap_dp = dp(item_list, 40)
print(knap_dp)

重さ40kg / 価格409万円
